## Situation and Task
Fix the problem that there are **large price discrepancies between the bond prices and HY curve**, especially on the short- and the long-end 

- This draws client challenge for both our pricing and curves product.
- **Existing solution**: the curve we built are 6%-coupon YTM bullet-equivalent curves. That is, after **normalizing for bond attributes** such as options, amount outstanding, coupon, etc, we fit the YTM using a **global log-linear model**.
    - The curve on the **short end is usually too rich**, especially for **lower rating or distressed bond**
    - After fitting the belly portion of the curve, the log-linear model **extrapolates on the long-end, rendering it too cheap**.

## Action

Develop a parametric model for the default term-structure, explicitly accounting for the accelaration of cashflow due to default and also fractional recovery of par.

- **Initial response**: we set out a code review where I read thousands of lines of legacy codes. I did find several bugs along the way, but nothing relevant to the issue above.
- **Pivot**: After confirming there is no problem in the implementation, I **turn to the methodology**: after all, inverted curve in the short-end for YTM is normal for distressed bonds. That inspires me to **go back to first principle** and explicitly model the the **time-to-default and the resulting acceleration of cashflow, taking into account recovery assumption**. Once we have the risky discount factors, it is simple bond maths to arrive at YTM curves to plot.
- **Further technical details**
    - The **risk-free discount factor** is obtained via **bootstrapping the government bonds in the corresponding currency**. Note that for some, e.g. EUR and CHF, the discount factor **may not be downward sloping**.
    - We tried specifying piecewise **constant hazard rate** for the survival probability, but find that somewhat an **exponential spline** is better, especially for EUR and CHF to avoid negative hazard rate.
    - The further appeal of **exponential spline** is that, solving for coefficients are just **solving a linear regression with number of data points being the number of bonds**; see more details in this [notebook](../../fixed-income/instruments/gsac/credit-pricing-and-analytics.ipynb)
    - We fit **all the bonds within the same cap structure to enlarge the data**: the **same survival probability** term structure and **different recoveries according to seniority**. To maintain a soft constraint on recoveries across cap structures, we specify **priors of recovery in the optimization**.

## Resolution
We are able to **reduce pricing errors**, and **mis-priced bonds typically revert more quickly**.

- **Next steps**: But the shape of the YTM curve transformed from the risky discount curve does not appeal to product owner's eyes, especially for those negative-yielding currencies.
- **Take-away**: to solve problems, one needs to **go back to first principles**, and clearly identify assumptions that we can do without.